In [1]:
import pandas as pd
import json
import re
import numpy as np

In [29]:
from unidecode import unidecode
from num2words import num2words
from nltk.stem import WordNetLemmatizer 
from fuzzywuzzy import fuzz

def text_cleaning(text):
    
    text = text.lower()
    text = text.replace("art.", "art ")
    
    text = unidecode(text)

    # replace spaces
    text = re.sub("\s\s+", ' ', text)

    # parenthesis
    text = re.sub("\(.+?\)", ' ', text)

    # page number
    text = re.sub("-\s[0-9]+\s-", '', text)

    # special characters
    text = re.sub("[^\w\s]", ' ', text)
    
    # replace spaces
    text = re.sub("\s\s+", ' ', text)
    
    text = text.split(" ")
    text = [num2words(word, lang = 'ro') if word.isnumeric() else word for word in text]
    
    text = " ".join(text)
    
    text = unidecode(text)

    return text

def fuzzy_search2(phrase, corpus):
    #phrase = text_cleaning(phrase)
    value = fuzz.token_set_ratio(corpus, phrase)
    if value >= 80:
        return True
    
    return False

def get_timestamp_for_steno(steno, df_transcript):
    
    # Todo: daca cubul are ca maxim valori negative atunci ignori
    
    #print("sample")
    
    # get is in corpus
    df_transcript['in_corpus'] = df_transcript['text'].map(lambda x: fuzzy_search2(x, steno))
    
    # count consecutive True values
    g = df_transcript['in_corpus'].ne(df_transcript['in_corpus'].shift()).cumsum()
    df_transcript['Count'] = df_transcript.groupby(g)['in_corpus'].transform('size') * np.where(df_transcript['in_corpus'], 1, -1)
    
    # get max of consecutive True values
    max_consecutive = df_transcript['Count'].max()
    
    # filter the cube
    # Todo: take first max_consecutive
    df_cube = df_transcript[df_transcript['Count'] == max_consecutive]
    
    # return start timestamp, end timestamp
    return (df_cube['start'].min(), df_cube['start'].max())

def get_timestamp_for_steno_2(steno, df_transcript):
    
    # get is in corpus
    transcript_text = df_transcript['text'].to_numpy()
    in_corpus = [fuzzy_search2(transcript, steno) for transcript in transcript_text]
    #df_transcript['in_corpus'] = df_transcript['text'].map(lambda x: fuzzy_search2(x, steno))
    df_transcript['in_corpus'] = in_corpus
    
    # count consecutive True values
    g = df_transcript['in_corpus'].ne(df_transcript['in_corpus'].shift()).cumsum()
    df_transcript['Count'] = df_transcript.groupby(g)['in_corpus'].transform('size') * np.where(df_transcript['in_corpus'], 1, -1)
    
    # get max of consecutive True values
    max_consecutive = df_transcript['Count'].max()
    
    # filter the cube
    df_cube = df_transcript[df_transcript['Count'] == max_consecutive]
    
    # asta e pentru fisier
    # return (df_cube['start'].min(), df_cube['start'].max())
    
    # asta e pentru vizualizare cub
    # return (df_cube['start'].min(), df_cube['end'].max(), df_cube)

    

In [3]:
# Get steno
df_steno = pd.read_json("/Users/andreibobes/Documents/Projects/NLP2/Rosen_Project/rosen-dataset/sample_data/pdf_extractions/seg_steno_86_21_decembrie_2022.json")
df_steno = df_steno.T

In [4]:
# get transcript
file = open("/Users/andreibobes/Documents/Projects/NLP2/Rosen_Project/rosen-dataset/sample_data/transcriptions/21_Decembrie_202200001.json")
data = json.load(file)
df_transcript = pd.DataFrame.from_dict(data['segments'])

In [5]:
# clean steno
df_steno['text'] = df_steno['text'].map(text_cleaning)
df_transcript['text'] = df_transcript['text'].map(text_cleaning)

In [ ]:
#df_steno['timestamp'] = df_steno['text'].apply(lambda x: get_timestamp_for_steno(x, df_transcript))

In [6]:
%%time
text = df_steno['text'].to_numpy()

CPU times: user 532 µs, sys: 17 µs, total: 549 µs
Wall time: 560 µs


In [30]:
%%time
timestamp = [get_timestamp_for_steno_2(steno, df_transcript) for steno in text]

CPU times: user 3min 34s, sys: 3.82 s, total: 3min 38s
Wall time: 3min 38s


In [31]:
df_steno['timestamp'] = timestamp

In [32]:
df_steno.to_csv('steno_21_decembrie_2022_timestamps.csv')

In [22]:
steno = text[60]

In [23]:
steno

' multumesc domnule presedinte vreau sa va spun ca domnul de la usr colegul meu a fost total defazat nu are nicio legatura cu initiativa legislativa noi aici vorbim de pana la urma de doua sute saizeci si doi de centre locale ale agentiei de plati si interventii in agricultura si de patruzeci si doi de judete care trebuie sa ajunga la nivelul salariilor celui din aparatul central vorbim de niste oameni care se ocupa de dimineata pana seara de subventiile pe care le primesc cei din agricultura si atunci ar trebui sa fie normal ca acesti oameni sa primeasca salariul similar ca cei din aparatul central despre asta este vorba stimate coleg nu despre ce ati batut dumneavoastra campii in aceste doua minute va multumesc noi vom vota pentru partidul social democrat pentru acest proiect si pentru aceasta initiativa legislativa va multumesc '

In [24]:
start, end, df = get_timestamp_for_steno_2(steno, df_transcript)

In [25]:
df['start'] = df['start'].map(lambda x: divmod(x, 60))
df['end'] = df['end'].map(lambda x: divmod(x, 60))

/var/folders/b0/nrwjbmq57vl86g88cs5d5zv80000gn/T/ipykernel_55569/3431322156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start'] = df['start'].map(lambda x: divmod(x, 60))
/var/folders/b0/nrwjbmq57vl86g88cs5d5zv80000gn/T/ipykernel_55569/3431322156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end'] = df['end'].map(lambda x: divmod(x, 60))


In [26]:
df

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,in_corpus,Count
408,408,455450,"(75.0, 54.5)","(75.0, 56.5)",multumesc domnule presedinte,"[29960, 15918, 10018, 66, 11, 3285, 77, 2271, ...",0.0,-0.146330,1.447471,2.147767e-08,True,7
409,409,455450,"(75.0, 56.5)","(76.0, 6.5)",vreau sa va spun ca domnul de la usr colegul ...,"[691, 265, 1459, 15446, 371, 4089, 37038, 2195...",0.0,-0.146330,1.447471,2.147767e-08,True,7
410,410,455450,"(76.0, 6.5)","(76.0, 19.5)",noi aici vorbim de pana la urma de doua sute ...,"[883, 72, 257, 8787, 4245, 65, 332, 368, 280, ...",0.0,-0.146330,1.447471,2.147767e-08,True,7
411,411,457950,"(76.0, 19.5)","(76.0, 27.5)",vorbim de niste oameni care se ocupa de dimin...,"[12231, 65, 332, 368, 3867, 10355, 975, 277, 3...",0.0,-0.095032,1.497872,8.494594e-08,True,7
412,412,457950,"(76.0, 27.5)","(76.0, 35.5)",si atunci ar trebui sa fie normal ca acesti o...,"[41820, 412, 11228, 594, 2192, 65, 3077, 15446...",0.0,-0.095032,1.497872,8.494594e-08,True,7
413,413,457950,"(76.0, 35.5)","(76.0, 41.5)",despre asta este vorba stimata coleg nu despr...,"[3885, 3712, 43405, 4065, 4245, 4231, 342, 332...",0.0,-0.095032,1.497872,8.494594e-08,True,7
414,414,460150,"(76.0, 41.5)","(76.0, 52.5)",va multumesc noi vom vota pentru partidul soc...,"[691, 4089, 14077, 15918, 10018, 66, 11, 22447...",0.0,-0.136588,1.467337,1.837085e-08,True,7


In [ ]:
end

In [ ]:
df.to_csv("new_transcript.csv")